In [1]:
import os

#extracts the json directory
files = [file for file in os.listdir('Data\Vehicle Data') 
if os.path.isfile(os.path.join('Data\Vehicle Data',file))==True]
json_files = [os.path.join('Data\Vehicle Data',file) for file in files if file.split('.')[1]=='json']
json_files

['Data\\Vehicle Data\\bicycle_tricycle.json',
 'Data\\Vehicle Data\\buses.json',
 'Data\\Vehicle Data\\cars_SUV.json',
 'Data\\Vehicle Data\\jeepney.json',
 'Data\\Vehicle Data\\largeTrucks.json',
 'Data\\Vehicle Data\\motorcycles_scooters.json',
 'Data\\Vehicle Data\\motorela.json',
 'Data\\Vehicle Data\\smallTrucks.json',
 'Data\\Vehicle Data\\taxi.json']

In [2]:
import logic as lc

vtype_dict = {}
for i in json_files:
    vtype = lc.json_loader(i)
    vtype_dict[vtype['id']] = vtype

vtype_dict.keys()

dict_keys(['bicycle_tricycle', 'buses', 'cars_SUV', 'jeepney', 'largeTrucks', 'motorcycles_scooters', 'motorela', 'smallTrucks', 'taxi'])

In [3]:
import pandas as pd
import openpyxl

#Create dataframe of probability distribution from TVC Data
wb = openpyxl.load_workbook('Data\Intersection Data\Corrales - JR Borja\TVC Summary.xlsx',data_only=True)
sheets = wb.sheetnames
all_dfs=[]
for sheet in sheets:
    ws = wb[sheet]

    column_names=ws['B1:K1']
    data = ws['B2:K17']

    all_data = []
    for row in data:
        rows=[]
        for cell in row:
            rows.append(cell.value)
        all_data.append(rows)

    columns=[]
    for row in column_names:
        for cell in row:
            columns.append(cell.value)
    
    df = pd.DataFrame(all_data, columns=columns)
    all_dfs.append(df)

main_df = pd.concat(all_dfs).fillna(0).reset_index(drop=True).iloc[:,0:-1]  #drop others column
distribution_df = main_df.sum()/main_df.sum().sum()
distribution_df #probability distribution

cars_SUV                0.315631
motorcycles_scooters    0.238142
taxi                    0.096226
buses                   0.000798
smallTrucks             0.013210
largeTrucks             0.000975
bicycle_tricycle        0.003576
jeepney                 0.050330
motorela                0.281112
dtype: float64

In [84]:
from pathlib import Path
reference = lc.json_loader(Path(r'Data\Intersection Data\Corrales - JR Borja\reference_dict.json')) #used to for clarity purposes

lane_link={ 
    reference['North_a']    :[reference['South_f'], reference['West_f']],
    reference['West_a']     :[reference['South_f'], reference['East_f']],
    reference['South_a']    :[reference['East_f'], reference['North_f']],
    reference['East_a']     :[reference['North_f'], reference['West_f']]
}
equal_route_distribution_probability =1/sum([len(lane) for lane in lane_link.values()])

route_distribution_command_list = []
count = 0
route_distribution_command_list.append('<routeDistribution id="routedist1">')
for lane_approach in lane_link:
    for lane_exit in lane_link[lane_approach]:
        command = f'    <route id="route{count}" edges="{lane_approach} {lane_exit}" probability="{equal_route_distribution_probability}"/>'
        route_distribution_command_list.append(command)
        count += 1

route_distribution_command_list.append('</routeDistribution>')
route_distribution_command_list

['<routeDistribution id="routedist1">',
 '    <route id="route0" edges="785902529 732022517" probability="0.125"/>',
 '    <route id="route1" edges="785902529 -785902540#1" probability="0.125"/>',
 '    <route id="route2" edges="785902540#0 732022517" probability="0.125"/>',
 '    <route id="route3" edges="785902540#0 785902540#2" probability="0.125"/>',
 '    <route id="route4" edges="-732022517 785902540#2" probability="0.125"/>',
 '    <route id="route5" edges="-732022517 -785902529" probability="0.125"/>',
 '    <route id="route6" edges="-785902540#2 -785902529" probability="0.125"/>',
 '    <route id="route7" edges="-785902540#2 -785902540#1" probability="0.125"/>',
 '</routeDistribution>']

In [85]:
import numpy as np
probability_dict={}

BEGIN = 0
END = 3600

for idx, value in enumerate(distribution_df):
    probability_dict[distribution_df.index.values[idx]] = float(np.round(value, 3))

vtype_command_list = []
#compile dict and json to vtype command
for prob_id in probability_dict:
    #vType command maker
    keys = list(vtype_dict[prob_id].keys())
    values = list(vtype_dict[prob_id].values())
    key_value_string = ["<vType"]+[f'{key}="{value}"' for key,value in zip(keys,values)] + ["/>"]
    command = " ".join(key_value_string)
    vtype_command_list.append(command)

    #Flow command maker
    probability = probability_dict[prob_id]
    vtype_id = vtype_dict[prob_id]['id']
    flow = f'    <flow type="{vtype_id}" id="{vtype_id}" probability="{probability_dict[prob_id]}" begin="{BEGIN}" end="{END}" route="routedist1"/>'
    vtype_command_list.append(flow)
vtype_command_list

['<vType accel="0.8" decel="4.5" sigma="0.5" length="5.0" minGap="2.5" maxSpeed="30.0" guiShape="passenger" color="0,0,255" id="cars_SUV" />',
 '    <flow type="cars_SUV" id="cars_SUV" probability="0.316" begin="0" end="3600" route="routedist1"/>',
 '<vType accel="1.0" decel="4.5" sigma="0.5" length="2.0" minGap="0.5" maxSpeed="30.0" guiShape="bicycle" color="255,0,0" id="motorcycles_scooters" />',
 '    <flow type="motorcycles_scooters" id="motorcycles_scooters" probability="0.238" begin="0" end="3600" route="routedist1"/>',
 '<vType accel="1.0" decel="4.5" sigma="0.5" length="2.0" minGap="0.5" maxSpeed="30.0" guiShape="passenger/sedan" color="85,176,73" id="taxi" />',
 '    <flow type="taxi" id="taxi" probability="0.096" begin="0" end="3600" route="routedist1"/>',
 '<vType accel="0.4" decel="2.0" sigma="0.5" length="15.0" minGap="2.5" maxSpeed="30.0" guiShape="bus" color="255,255,0" id="buses" />',
 '    <flow type="buses" id="buses" probability="0.001" begin="0" end="3600" route="ro

## Create Distribution XML on Simulation Training Folder 

In [73]:
PATH = Path(r'Simulation_Environment\Simulation Training')

with open(os.path.join(PATH,'routeDistribution.xml'), 'w') as f:
    f.write("<routes> \n")

    for command in route_distribution_command_list:
        f.write(f'  {command}\n')

    for command in vtype_command_list:
        f.write(f'  {command}\n')
    
    f.write('</routes>')

In [83]:
import numpy as np

for i in range(10):
    print(np.random.randint(0,2))

1
0
0
0
0
1
1
0
1
0
